In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

PETFINDER_API_KEY = os.getenv("PETFINDER_API_KEY")
PETFINDER_SECRET_KEY = os.getenv("PETFINDER_SECRET_KEY")

In [16]:
import requests

auth_url = "https://api.petfinder.com/v2/oauth2/token"
auth_data = {
    "grant_type": "client_credentials",
    "client_id": PETFINDER_API_KEY,
    "client_secret": PETFINDER_SECRET_KEY,
}
auth_response = requests.post(auth_url, data=auth_data)
auth_response.raise_for_status()
access_token = auth_response.json()["access_token"]

In [17]:
params = {
    "limit": 100,
    "type": "cat",
    "status": "adoptable",
    "location": "Fremont, CA",
}

headers = {
    "Authorization": f"Bearer {access_token}",
}

url = "https://api.petfinder.com/v2/animals"

first_page = requests.get(url, headers=headers, params=params)
first_page.raise_for_status()
first_page_data = first_page.json()

if not first_page_data["pagination"]:
    raise ValueError("No pagination data found in the first page.")
total_pages = first_page_data["pagination"]["total_pages"]

first_page_data

{'animals': [{'id': 77830183,
   'organization_id': 'CA912',
   'url': 'https://www.petfinder.com/cat/yale-77830183/ca/milpitas/humane-society-silicon-valley-ca912/?referrer_id=bdb3e282-b9fb-4460-9be2-ab2573475b0b&utm_source=api&utm_medium=partnership&utm_content=bdb3e282-b9fb-4460-9be2-ab2573475b0b',
   'type': 'Cat',
   'species': 'Cat',
   'breeds': {'primary': 'Domestic Medium Hair',
    'secondary': None,
    'mixed': True,
    'unknown': False},
   'colors': {'primary': None, 'secondary': None, 'tertiary': None},
   'age': 'Baby',
   'gender': 'Male',
   'size': 'Small',
   'coat': None,
   'attributes': {'spayed_neutered': False,
    'house_trained': False,
    'declawed': False,
    'special_needs': False,
    'shots_current': True},
   'environment': {'children': None, 'dogs': None, 'cats': None},
   'tags': [],
   'name': 'Yale',
   'description': 'Hello! Are you looking for a social kitty who loves getting showered with attention? Look no further, because I&amp;#39;m your...

In [23]:
curr_page = 1
data_source = []

In [24]:
import time

while curr_page <= total_pages:
    params["page"] = curr_page
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    data = response.json()
    
    if not data["animals"]:
        raise(f"Error: {data['message']}")

    total_pages = data["pagination"]["total_pages"]
    
    data_source.extend(data["animals"])
    curr_page += 1
    time.sleep(2)

In [ ]:
with open("cats.json", "w") as f:
    import json
    json.dump(data_source, f)